In [2]:
# !sudo pip install --upgrade transformers
# !sudo pip install fuzzywuzzy

In [3]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
#from Gpt4_OpenAI_Prompt import system_msg,context,postprocess
#from prompt1 import system_msg
import pandas as pd
import transformers
from jinja2 import Template
from fuzzywuzzy import process
print(transformers.__version__)

4.35.0


/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
print(torch.cuda.is_available())
#print(torch.cuda.get_device_name())

False


In [5]:
context = {
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
                         "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
                        {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
                        {"ENTITY": "contribution_to_growth",
                         "other names": ["contribution to growth", "growth", "grown"]},
                        {"ENTITY": "kda_transactional",
                         "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
                        {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
                        {"ENTITY": "correlation",
                         "other names": ["associate", "associated", "association", "associations", "correlate",
                                         "correlated",
                                         "correlation", "correlations", "relate", "related", "relation", "relations",
                                         "relationship",
                                         "relationships"]}
                        ],
    "DATE VARIABLE": [
        {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
}

In [6]:
def get_context_df():
    df_final = pd.DataFrame()
    for i in context:
        df = pd.DataFrame(context[i])
        df["Type"]  = i
        df_final = pd.concat([df,df_final],ignore_index=True)
    return df_final

In [10]:
# ################################################################################
# # bitsandbytes parameters
# ################################################################################

# # Activate 4-bit precision base model loading
# use_4bit = True

# # Compute dtype for 4-bit base models
# bnb_4bit_compute_dtype = "float32"

# # Quantization type (fp4 or nf4)
# bnb_4bit_quant_type = "fp4"

# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# # Activate nested quantization for 4-bit base models (double quantization)
# use_nested_quant = False

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )
# # Load the entire model on the GPU 0
# device_map = {"": 0}


In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# model_mistral = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
#     device_map=device_map)
model_mistral = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer_mistral = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def gereneate_output_mistral(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer_mistral(prompt, return_tensors="pt")#.to("cuda")
    #model.eval()
    response = tokenizer_mistral.decode(model_mistral.generate(**model_input, max_length=1024*2)[0], skip_special_tokens=True)
    return response

In [7]:
def get_user_question_for_open_ai_prompt(question,context):
    return """Now Based on this following context: \n """+ str(context) + "\n return a response for this question:\n" + question + "\n"
    #return "\n Now detect from this question:\n" + question + "\n"

def compose_prompt_open_ai_prompt(question,base_prompt,context):
    return f"""[INST]<<SYS>>
            {base_prompt}
            <</SYS>>
            {get_user_question_for_open_ai_prompt(question,context)}
            [/INST]
            [_separator_]"""


def get_user_question_for_open_chat(question,context):
    return """Now Based on this following context: \n """+ str(context) + "\n return a response for this question:\n" + question + "\n"
    #return "\n Now detect from this question:\n" + question + "\n"

def compose_prompt_open_chat(question,base_prompt,context):
    return f"""<s>[IDENTITY]
            {base_prompt}
            [/IDENTITY]</s>
            [INST]
            {get_user_question_for_open_chat(question,context)}
            [/INST]
            """


In [8]:
def find_best_match_from_context(word, df):
    matches = process.extractOne(word, df.values.flatten())
    max_match, score = matches
    index = df.values.flatten().tolist().index(max_match)
    row_index, col_index = divmod(index, df.shape[1])
    entity_name = df.iat[row_index, col_index]
    return df.iloc[row_index].to_dict()

In [9]:
MEASURE_LIST = sum([[i["ENTITY"]]+i["other names"] for i in context["MEASURE"]],[])
DIMENSION_LIST = sum([[i["ENTITY"]]+i["other names"] for i in context["DIMENSION"]],[])
FILTER_LIST = sum([[i["ENTITY"]]+i['other names'] for i in context["FILTER"]],[])

In [14]:
MEASURE_PROMPT = f'''

Task: Extract Metric Constraints and Measure Entities

Context: The context includes a list of measure entities that you consider as your MEASURE_ENTITIES_LIST

Instructions:

1. Given a question, determine if any measure entity from the MEASURE_ENTITIES_LIST is highly matched in the question.
2. Measure entity match should happen only in between the MEASURE_ENTITIES_LIST and the n-grams in the question.
2. If a measure entity is matched, check if there is any comparison intent present in the question only on that matched entity, such as "greater than," "less than," "equals to," or "in between."
3. For the matched measure entity, extract the following information:
    The matched measure entity in the question itself.
    The comparison operator (if present) in the form of "<," ">, "<=," ">="," or "equals to," or "in between" for range comparisons.
    The comparison value, which should be a numerical or scalar value (for comparison operators) or a range in the format "X to Y" (for "in between").
4. Remember, if no measure entity is highly matched with n-grams in the question , provide a response indicating that no measure entity from the provided context list is matched in the question.

5. Return the following attributes for each detected measure entity in a JSON list format:

'''


output_format = """
[
{
"ENTITY": (The specific measure entity matched from the MEASURE_ENTITIES_LIST.),
"COMPARISON OPERATOR": (only "=" or ">" or "<" or "<=" or ">=" whichever is detected else null),
"COMPARISON VALUE": (scalar value, If detected, else null)
}
]

**Few shot training***

**Example 1**
MEASURE_ENTITIES_LIST = ["Sales","purchase value"]
Question : "Show me the sale of phone when it has purchase value greater than 10K"

Output: 

[
{
"ENTITY": "sale",
"COMPARISON OPERATOR": "null",
"COMPARISON VALUE": "null"
},
{
"ENTITY": "purchase value",
"COMPARISON OPERATOR": ">",
"COMPARISON VALUE": "10K"
}
]
Explanation: In the question "sale" and "purchase value" entities are highly matched with provided context MEASURE_ENTITIES_LIST = ["Sales","purchase value"]
there is no comparison intent presnet for "Sales"
For "purchase value"  greater than inent is presnt and 10K is scaler value

**Example 2**
MEASURE_ENTITIES_LIST = ["Sales","purchase value"]
Question : "when was the first time sales of segments was 0"
Output: 

[
{
"ENTITY": "sales",
"COMPARISON OPERATOR": "=",
"COMPARISON VALUE": "0"
}
]
"EXPLANATION": In the question "sales" is matched with "Sales" from MEASURE_ENTITIES_LIST, the entity 'sales' is explicitly mentioned with a comparison operator '=' and a comparison value '0' in the given question."

**Example 3**
MEASURE_ENTITIES_LIST = ["purchase value","discounts","distance"]
Question : "purchase value from 100 to 200"
Output: 

[
{
"ENTITY": "purchase value",
"COMPARISON OPERATOR": ">=",
"COMPARISON VALUE": "100"
},
"ENTITY": "purchase value",
"COMPARISON OPERATOR": "<=",
"COMPARISON VALUE": "200"
}
]
"EXPLANATION":  In the question "purchase value" is matched with "purchase value" from MEASURE_ENTITIES_LIST, "The entity 'purchase value' is specified with a comparison operator '<=' and a comparison value '200'."
and "The entity 'purchase value' is also specified with a comparison operator '>=' and a comparison value '100'."

**Example 4**
MEASURE_ENTITIES_LIST = ["Medicine weight","Tablet cost"]
Question : "what are the medicine weight in between 10M to 20M"
Output: 
[
{
"ENTITY": "medicine weight",
"COMPARISON OPERATOR": ">",
"COMPARISON VALUE": "10M"
},
"ENTITY": "medicine weight",
"COMPARISON OPERATOR": "<",
"COMPARISON VALUE": "20M"
}
]

"EXPLANATION": "In the question "medicine weight" is matched with "medicine weight" this entity  is specified with a comparison operator '<=' and a comparison value '20M'."
"also the entity 'Medicine weight' is specified with a comparison operator '>=' and a comparison value '10M'."

**Example 5**

MEASURE_ENTITIES_LIST = ["Medicine weight","Tablet cost"]
Question : "what are the discounts in between 10M to 20M"
Output: 
[
{
"ENTITY": "null",
"COMPARISON OPERATOR": "null",
"COMPARISON VALUE": "null"
}
]
"EXPLANATION": "No measure entity from the provided context list is highly matched in the question."


**Example 6**

MEASURE_ENTITIES_LIST = ["Sales","purchase value"]
Question : "what are the discounts in India"
Output: 
[
{
"ENTITY": "null",
"COMPARISON OPERATOR": "null",
"COMPARISON VALUE": "null"
}
]
"EXPLANATION": "No measure entity from the provided context MEASURE_ENTITIES_LIST list is matched in the question."

"""


MEASURE_PROMPT = MEASURE_PROMPT+output_format

In [9]:
ADJECTIVE_and_TONE = f"""
Task: Extract Adjectives, Determine Intent (TONE), Return RANK ADJECTIVE, and Scalar (RANK)

Instructions:

Consider these list of dimension entities {DIMENSION_LIST} as DIMENSION_ENTITIES_LIST
For each dimension entity detected in the give question, proceed to determine if there is any Adjective intent present within the question, Adjectives can include terms like "least" "highest", etc
Determine the intent (TONE) of the adjective. The intent can be only be either positive or negative.
If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
If the adjective intent is positive, always set RANK ADJECTIVE to "TOP" and TONE to "positive."
If the adjective implies a quantity (e.g., "how many," "top 3"), extract that numerical scalar value as "RANK"
Return the following attributes for each detected dimension entity in a JSON list format: """
output_format="""
[
{
"ADJECTIVE": whichever is detected else null,
"TONE": "positive" or "negative"
"ENTITY": The specific dimension entity matched from the DIMENSION_ENTITIES_LIST.,
"RANK ADJECTIVE": "TOP" or "BOTTOM",
"RANK": "if scalar value detected else null"
}
]
"""
ADJECTIVE_and_TONE = ADJECTIVE_and_TONE+output_format


In [10]:
DIMENSION_PROMPT = f"""
Task: Extract Adjectives, Determine Intent (TONE), Return RANK ADJECTIVE, and Scalar (RANK)
Context: The context includes a list of dimension entities that you consider as your DIMENSION_ENTITIES_LIST

Instructions:

1. Given a question, determine if any dimension entity from the DIMENSION_ENTITIES_LIST is detected in the question.
2. If a dimension entity is detected, proceed to determine if there is any Rank Adjective intent present within the question as RANK ADJECTIVE, Rank Adjectives can include terms like "top" "best","least" etc
3. From the Rank ADJECTIVE if adjective implies a quantity (e.g., "least 10," "top 3"), extract that numerical scalar value as "RANK". If there is no scaler is identified assume "RANK" as 1
4. Return the following attributes for each detected dimension entity in a JSON list format: """

output_format="""
[
{
"ENTITY": The specific dimension entity matched from the DIMENSION_ENTITIES_LIST.,
"RANK ADJECTIVE": "If detected else null",
"RANK": "if scalar value detected "
}
]

**Few shot training***

1. *Example 1**
DIMENSION_ENTITIES_LIST = ["parts","Country"]
Question : "Show me top 10 parts in the Dubai"

Output: 
[
{
"ENTITY": "parts",
"RANK ADJECTIVE": "top",
"RANK": "10"
}
]

Explanation: In the question parts is detected as entity as it is there in DIMENSION_ENTITIES_LIST, top is detected as RANK ADJECTIVE, and RANK is is 10 which tells howmany tops' should consider.
 

2. *Example 2**
DIMENSION_ENTITIES_LIST = ["parts","Country"]
Question : "Show me top 10 segments in the Dubai"

Output: 
[
{
"ENTITY": "null",
"RANK ADJECTIVE": "null",
"RANK": " null"
}
]
Explanation : None of the entities from DIMENSION_ENTITIES_LIST is there in question, hence no need to detect any thing. Just return null

3. *Example 3**
DIMENSION_ENTITIES_LIST = ["parts","Country"]
Question : "Show me worst performing conutry based on sales"

Output: 
[
{
"ENTITY": "conutry",
"RANK ADJECTIVE": "worst",
"RANK": "1"
}
]
Explanation: "conutry" is detected in question from the given DIMENSION_ENTITIES_LIST, and also "RANK ADJECTIVE" is "worst" and as there is no scaler even thought the Adjective is detected RANK was defaulted to 1

4. *Example 4**
DIMENSION_ENTITIES_LIST = ["parts"]
Question : "Show me worst performing conutry"

Output: 
[
{
"ENTITY": "null",
"RANK ADJECTIVE": "null",
"RANK": "null"
}
]
Explanation : As there are no enities detected in the question from the given context DIMENSION_ENTITIES_LIST, hence no need to detect for any other entities in the question. Just return null

5. *Example 5**
DIMENSION_ENTITIES_LIST = ["parts"]
Question : "sales greater than 100"

Output: 
[
{
"ENTITY": "null",
"RANK ADJECTIVE": "null",
"RANK": "null"
}
]
Explanation : As there are no enities detected in the question from the given context DIMENSION_ENTITIES_LIST, hence no need to detect for any other entities in the question. Just return null



6. *Example 6**
DIMENSION_ENTITIES_LIST = ["parts","segments"]
Question : "top5 segments and bottom 15 parts"

Output: 
[
{
"ENTITY": "parts",
"RANK ADJECTIVE": "bottom",
"RANK": "15"
},
{
"ENTITY": "segments",
"RANK ADJECTIVE": "top",
"RANK": "5"
}
]
Explanation: "parts" and "segments" provided in the context DIMENSION_ENTITIES_LIST, RANK ADJECTIVES and RANK for "parts" and "segments" were  detected.

7. *Example 7**
DIMENSION_ENTITIES_LIST = ["parts"]
Question : "top5 segments and bottom 15 parts"

Output: 
[
{
"ENTITY": "parts",
"RANK ADJECTIVE": "bottom",
"RANK": "15"
}
]

Explanation: "parts"  is provided in the context DIMENSION_ENTITIES_LIST and it is only the matched entity in the question, hence RANK ADJECTIVES and RANK for "parts" were detected. Even though there is some Adjective is there for Segment here no need to detect as Segment Entity is not provided in the context.

"""

In [11]:
##*DONE*##
FILTER_PROMPT = f'''
Task: Extract Filter Entities
Analyze the provided question.
Consider only filter entity detected in the question, determine if it's intended to be included or excluded.
Set "EXCLUDE" to "True" if the detected filter entity is intended to be excluded else Set "EXCLUDE" to "False"

Return the only the following attributes for each detected filter entity in a json list:'''

output_format = """

{
"EXCLUDE": ("True" or "False"),
"ENTITY": (The specific filter entity matched from the FILTER_ENTITIES_LIST.)
}

"""
FILTER_PROMPT = FILTER_PROMPT+output_format

In [12]:
##*DONE*##
FILTER_PROMPT_Context = f'''
Task: Extract Filter Entities
Consider these list of filter entities {FILTER_LIST} as FILTER_ENTITIES_LIST.
Analyze the provided question.
Consider only filter entity detected in the question from FILTER_ENTITIES_LIST, determine if it's intended to be included or excluded.
Set "EXCLUDE" to "True" if the detected filter entity is intended to be excluded else Set "EXCLUDE" to "False"

Return the only the following attributes for each detected filter entity in a json list:'''

output_format = """

{
"EXCLUDE": ("True" or "False"),
"ENTITY": (The specific filter entity matched from the FILTER_ENTITIES_LIST.)
}

"""
FILTER_PROMPT_Context = FILTER_PROMPT_Context+output_format

In [13]:
##*DONE*##

MEASURE_CONSTRAINT = f'''
Task: Extract metric constraints and Measure Entities
1. Consider these list of measure entities {MEASURE_LIST} as MEASURE_ENTITIES_LIST.
2. For each measure entity detected in the give question, proceed to determine if there is any comparison intent present within the question, such as "greater than," "less than," or "equals to."
3. For all such only matched measure entities extract the comparison operator (such as >,<,<=,>=, =) as "COMPARISON OPERATOR" (if a comparison intent is present else null).
4. For all such only matched measure entities extract the "COMPARISON VALUE" which should be a numerical or scalar value which will be operationg on "COMPARISON OPERATOR".
5. Return the following attributes for each detected measure entity in a JSON list format:'''

output_format = """
[
{
"ENTITY": (The specific measure entity matched from the MEASURE_ENTITIES_LIST.),
"COMPARISON OPERATOR": (only "=" or ">" or "<" or "<=" or ">=" whichever is detected else null),
"COMPARISON VALUE": (scalar value, If detected, else null)
}
]
"""
MEASURE_CONSTRAINT = MEASURE_CONSTRAINT+output_format

In [14]:
DERIVED_MEASURE = """Task: Extract the Derived Measure from User Queries

Understand the following Derived Measure definitions:
1. "Ratio": Represents a Derived Measure and can also be referred to as "ratio," "share," "contribution," "percentage," "proportion," or "contributing."
2. "Why": Represents a Derived Measure and has synonyms such as "why," "cause of," "reason for," and "diagnose."
3. "Contribution to Growth": Denotes a Derived Measure and includes synonyms like "contribution to growth," "growth," or "grown."
4. "KDA Transactional": Refers to a Derived Measure with synonyms including "kda," "key drivers," "key driver," "drivers," or "driver."
5. "Growth Rate": Represents a Derived Measure with synonyms like "growth rate," "growth," or "grown."
6. "Correlation": Denotes a Derived Measure and has synonyms such as "associate," "associated," "association," "associations," "correlate," "correlated," "correlation," "correlations," "relate," "related," "relation," "relations," "relationship," or "relationships."

Your task is to examine the user's question to identify the Derived Measure concepts. 
If detected, find the specific targeted derived measure mentioned in the question and return it as the Derived Measure.

Return:
- Derived Measure: (if identified)
- ENTITIES: ["ENTITY1", "ENTITY2"] (if specified in relation to the Derived Measure)
"""

In [10]:
MEASURE = {
        "MEASURE": {
            "{{ entity_from_user_question }}": [
                {
                    "ENTITY": "{{ entity_from_context_data }}",
                    "MEASURE CONSTRAINT":[
                        {
                        "COMPARISON VALUE":"{{ comparison_value }}",
                        "COMPARISON OPERATOR":"{{ comparison_operator }}"
                        }

                    ]
                }
            ]
        }
    }

DIMENSION = {
    "DIMENSION": {
        "{{ entity_from_user_question }}": [
            {
                "ENTITY": "{{ entity_from_context_data }}",
                "ADJECTIVE" : " {{ model_output_adjective }} ",
                "TONE": "{{ model_output_tone }}",
                "RANK": [
                    {
                        "RANK ADJECTIVE": "{{ model_output_rank_adjective }}",
                        "RANK VALUE": "{{ model_output_rank_value }}"
                    }
                ]
            }
        ]
    }
}

FILTER = {
    "FILTER": {
      "{{ entity_from_user_question }}": [
         {
            "ENTITY": "{{ entity_from_context_data }}",
            "EXCLUDE": "{{ exclude_value }}",
            "PARENT": "{{ parent_value }}"
         }
      ]
   }
}

In [11]:
def get_Mql(prompt_type,model_output,context_data): 
    if prompt_type=="MEASURE":
        template_content = MEASURE
        template_content = json.dumps(template_content)
        template_content=template_content.replace("{{ entity_from_user_question }}", model_output["ENTITY"])
        template_content=template_content.replace("{{ entity_from_context_data }}", context_data["ENTITY"])
        template_content=template_content.replace("{{ comparison_value }}", str(model_output["COMPARISON VALUE"]))
        template_content=template_content.replace("{{ comparison_operator }}", model_output["COMPARISON OPERATOR"])
        return json.loads(template_content)
    if prompt_type=="DIMENSION":
        template_content = DIMENSION
        template_content = json.dumps(template_content)
        template_content=template_content.replace("{{ entity_from_user_question }}", model_output["ENTITY"])
        template_content=template_content.replace("{{ entity_from_context_data }}", context_data["ENTITY"])
        template_content=template_content.replace("{{ model_output_adjective }}", model_output["ADJECTIVE"])
        template_content=template_content.replace("{{ model_output_tone }}", model_output["TONE"])
        template_content=template_content.replace("{{ model_output_rank_adjective }}", model_output["RANK ADJECTIVE"])
        template_content=template_content.replace("{{ model_output_rank_value }}", str(model_output["RANK"]))
        return json.loads(template_content)
    if prompt_type=="FILTER":
        template_content = FILTER
        template_content = json.dumps(template_content)
        template_content=template_content.replace("{{ entity_from_user_question }}", model_output["ENTITY"])
        template_content=template_content.replace("{{ entity_from_context_data }}", context_data["ENTITY"])
        template_content=template_content.replace("{{ exclude_value }}", str(model_output["EXCLUDE"]))
        template_content=template_content.replace("{{ parent_value }}",str(context_data["parent"]))
        return json.loads(template_content)
    if prompt_type=="DERIVED_MEASURE":
        template_content = DERIVED_MEASURE
        template_content = json.dumps(template_content)
        template_content=template_content.replace("{{ derived_measure_from_user_question }}", model_output["DERIVED MEASURE"])
        template_content=template_content.replace("{{ measure_from_user_question }}", model_output["ENTITY MEASURE"])
        template_content=template_content.replace("{{ measure_from_context }}", context_data["MEASURE"])
        template_content=template_content.replace("{{ comparison_value }}", str(model_output["COMPARISON VALUE"])) 
        template_content=template_content.replace("{{ comparison_operator }}", model_output["COMPARISON OPERATOR"])
        template_content=template_content.replace("{{ derived_measure_from_context }}", context_data["DERIVED MEASURE"])
        template_content=template_content.replace("{{ filter_list_from_user_question }}", model_output["FILTERS"])
        return json.loads(template_content)
    

In [12]:
context_df = get_context_df()
def get_mql_measure():
    pass
    
def get_mql_filter(filter_model_output):
    list_filter_mlqs = []
    for outputs in filter_model_output:
        context_data = find_best_match_from_context(outputs['ENTITY'],context_df)
        Mql= get_Mql(prompt_type="FILTER",model_output=outputs,context_data=context_data)
        list_filter_mlqs.append(Mql)
    filter_mql = {}
    for data in list_filter_mlqs:
        for key, value in data.items():
            if key in filter_mql:
                filter_mql[key].update(value)
            else:
                filter_mql[key] = value
    return filter_mql

def get_mql_dimenssion():
    pass

In [18]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("openchat/openchat_3.5")
# model = AutoModelForCausalLM.from_pretrained("openchat/openchat_3.5")

## Measure (Comparison Intent)

In [16]:
import re
def format_measure_response(output_measure):
    output = "["+output_measure.split("[_separator_]")[-2].replace("\n","").split("[")[1]
    return json.loads(output)


In [36]:
%%time
question = "show me sales greater than 100" 
output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,MEASURE_PROMPT,context=MEASURE_LIST))
print(f"MEASURE_PROMPT output is : {output_measure}")
output_measure_json = format_measure_response(output_measure)
print("Post processed output")
print(output_measure_json)
print("MEASURE_PROMPT Done")

context_data = find_best_match_from_context(output_measure_json[0]['ENTITY'],context_df)
Mql_measure = get_Mql(prompt_type="MEASURE",model_output=output_measure_json[0],context_data=context_data)
print("--------MQL after post processing-----------")
print(json.dumps(Mql_measure,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MEASURE_PROMPT output is : [INST]<<SYS>>
            

Task: Extract Metric Constraints and Measure Entities

Context: The context includes a list of measure entities that you consider as your MEASURE_ENTITIES_LIST

Instructions:

1. Given a question, determine if any measure entity from the MEASURE_ENTITIES_LIST is detected in the question.
2. If a measure entity is detected, check if there is any comparison intent present in the question, such as "greater than," "less than," "equals to," or "in between."
3. For the detected measure entity, extract the following information:
    The measure entity itself.
    The comparison operator (if present) in the form of "<," ">, "<=," ">="," or "equals to," or "in between" for range comparisons.
    The comparison value, which should be a numerical or scalar value (for comparison operators) or a range in the format "X to Y" (for "in between").
4. If no measure entity is detected, provide a response indicating that no measure entity from the pro

In [37]:
%%time
question = "show me sales not greater than 100" 
output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,MEASURE_PROMPT,context=MEASURE_LIST))
print(f"MEASURE_PROMPT output is : {output_measure}")
output_measure_json = format_measure_response(output_measure)
print("Post processed output")
print(output_measure_json)
print("MEASURE_PROMPT Done")

context_data = find_best_match_from_context(output_measure_json[0]['ENTITY'],context_df)
Mql_measure = get_Mql(prompt_type="MEASURE",model_output=output_measure_json[0],context_data=context_data)
print("--------MQL after post processing-----------")
print(json.dumps(Mql_measure,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MEASURE_PROMPT output is : [INST]<<SYS>>
            

Task: Extract Metric Constraints and Measure Entities

Context: The context includes a list of measure entities that you consider as your MEASURE_ENTITIES_LIST

Instructions:

1. Given a question, determine if any measure entity from the MEASURE_ENTITIES_LIST is detected in the question.
2. If a measure entity is detected, check if there is any comparison intent present in the question, such as "greater than," "less than," "equals to," or "in between."
3. For the detected measure entity, extract the following information:
    The measure entity itself.
    The comparison operator (if present) in the form of "<," ">, "<=," ">="," or "equals to," or "in between" for range comparisons.
    The comparison value, which should be a numerical or scalar value (for comparison operators) or a range in the format "X to Y" (for "in between").
4. If no measure entity is detected, provide a response indicating that no measure entity from the pro

In [39]:
%%time
question = "show me purchase value lower than 10k and higher than 5k"
output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,MEASURE_PROMPT,context=MEASURE_LIST))
print(f"MEASURE_PROMPT output is : {output_measure}")
output_measure_json = format_measure_response(output_measure)
print("Post processed output")
print(output_measure_json)
print("MEASURE_PROMPT Done")

context_data = find_best_match_from_context(output_measure_json[0]['ENTITY'],context_df)
Mql_measure = get_Mql(prompt_type="MEASURE",model_output=output_measure_json[0],context_data=context_data)
print("--------MQL after post processing-----------")
print(json.dumps(Mql_measure,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MEASURE_PROMPT output is : [INST]<<SYS>>
            

Task: Extract Metric Constraints and Measure Entities

Context: The context includes a list of measure entities that you consider as your MEASURE_ENTITIES_LIST

Instructions:

1. Given a question, determine if any measure entity from the MEASURE_ENTITIES_LIST is detected in the question.
2. If a measure entity is detected, check if there is any comparison intent present in the question, such as "greater than," "less than," "equals to," or "in between."
3. For the detected measure entity, extract the following information:
    The measure entity itself.
    The comparison operator (if present) in the form of "<," ">, "<=," ">="," or "equals to," or "in between" for range comparisons.
    The comparison value, which should be a numerical or scalar value (for comparison operators) or a range in the format "X to Y" (for "in between").
4. If no measure entity is detected, provide a response indicating that no measure entity from the pro

In [40]:
%%time
question = "show me segments which are greater than 100"
output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,MEASURE_PROMPT,context=MEASURE_LIST))
print(f"MEASURE_PROMPT output is : {output_measure}")
output_measure_json = format_measure_response(output_measure)
print("Post processed output")
print(output_measure_json)
print("MEASURE_PROMPT Done")

context_data = find_best_match_from_context(output_measure_json[0]['ENTITY'],context_df)
Mql_measure = get_Mql(prompt_type="MEASURE",model_output=output_measure_json[0],context_data=context_data)
print("--------MQL after post processing-----------")
print(json.dumps(Mql_measure,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MEASURE_PROMPT output is : [INST]<<SYS>>
            

Task: Extract Metric Constraints and Measure Entities

Context: The context includes a list of measure entities that you consider as your MEASURE_ENTITIES_LIST

Instructions:

1. Given a question, determine if any measure entity from the MEASURE_ENTITIES_LIST is detected in the question.
2. If a measure entity is detected, check if there is any comparison intent present in the question, such as "greater than," "less than," "equals to," or "in between."
3. For the detected measure entity, extract the following information:
    The measure entity itself.
    The comparison operator (if present) in the form of "<," ">, "<=," ">="," or "equals to," or "in between" for range comparisons.
    The comparison value, which should be a numerical or scalar value (for comparison operators) or a range in the format "X to Y" (for "in between").
4. If no measure entity is detected, provide a response indicating that no measure entity from the pro

### Open Chat

In [13]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name ="openchat/openchat_3.5"
# model_mistral = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
#     device_map=device_map)
model_open_chat = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer_open_chat = AutoTokenizer.from_pretrained(model_name)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
def gereneate_output_openchat(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer_open_chat(prompt, return_tensors="pt")#.to("cuda")
    #model.eval()
    response = tokenizer_open_chat.decode(model_open_chat.generate(**model_input, max_length=1024*4)[0], skip_special_tokens=True)
    return response

In [53]:
from tqdm import tqdm

In [ ]:
questions = pd.read_csv("Question_set_mql.csv")
model_output_list = []
for index in tqdm(range(questions.shape[0])):
    question=questions.iloc[index]["Question"]
    try:
        output_measure=gereneate_output_openchat(compose_prompt_open_chat(question,MEASURE_PROMPT,context=MEASURE_LIST))
        output_ = output_measure.split("[SOL]")[-1].split("[/SOL]")[0].strip()
        if output_!="":
            output_ = json.loads(output_measure.split("[SOL]")[-1].split("[/SOL]")[0])
        else:
            output_ = [{'ENTITY': None, 'COMPARISON OPERATOR': None, 'COMPARISON VALUE': None}]
        print(f"{question}, {output_}")
        model_output_list.append(output_)
        time.sleep(5)
    except Exception as e:
        print(Exception)
        print(output_measure)
        print(question)
        print("#"*40)
        output_ = None
        model_output_list.append(output_)
        time.sleep(5)
questions["MEASURE"] = model_output_list

  0%|          | 0/40 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<class 'Exception'>
[IDENTITY]
            

Task: Extract Metric Constraints and Measure Entities

Context: The context includes a list of measure entities that you consider as your MEASURE_ENTITIES_LIST

Instructions:

1. Given a question, determine if any measure entity from the MEASURE_ENTITIES_LIST is highly matched in the question.
2. Measure entity match should happen only in between the MEASURE_ENTITIES_LIST and the n-grams in the question.
2. If a measure entity is matched, check if there is any comparison intent present in the question only on that matched entity, such as "greater than," "less than," "equals to," or "in between."
3. For the matched measure entity, extract the following information:
    The matched measure entity in the question itself.
    The comparison operator (if present) in the form of "<," ">, "<=," ">="," or "equals to," or "in between" for range comparisons.
    The comparison value, which should be a numerical or scalar value (for comparison operator

  2%|▎         | 1/40 [02:47<1:48:53, 167.53s/it]Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


discount rate of phone and binders, [{'ENTITY': None, 'COMPARISON OPERATOR': None, 'COMPARISON VALUE': None}]


  5%|▌         | 2/40 [04:45<1:27:43, 138.52s/it]Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


...

 48%|████▊     | 19/40 [56:04<1:01:12, 174.89s/it]Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


how many segments contributing to growth of sales in p3m vs pp, [{'ENTITY': None, 'COMPARISON OPERATOR': None, 'COMPARISON VALUE': None}]


 50%|█████     | 20/40 [58:25<54:56, 164.82s/it]  Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


how has sales trended in first week of 2021, [{'ENTITY': None, 'COMPARISON OPERATOR': None, 'COMPARISON VALUE': None}]


 52%|█████▎    | 21/40 [1:00:40<49:18, 155.69s/it]Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [77]:
%%time
question = "show me sales which are greater than 100"
output_measure=gereneate_output_openchat(compose_prompt_open_chat(question,system_msg,context=context))

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


CPU times: user 1h 20min 57s, sys: 14min 9s, total: 1h 35min 6s
Wall time: 12min 50s


In [79]:
print(output_measure)

[IDENTITY]
            You are an assistant that helps to map the user question to the context for a question answering system.
 You might also need to act as a time tagger expert to convert the date elements present in the question to a standard format and to find possible date ranges for the same.
Step 1: Identify the n-grams match between question and context
Map the n-gram or their lemma or their inflections from the question with the 'other names' in the passed context.
Always consider the longest n-gram match, not the sub-string.
If there are multiple matches for an n-gram with context, return all such ENTITY in response.
If you are returning any match which is not exactly present with the 'other names', make sure that it is a noun phrase and there is a high similarity between the match and the matched "ENTITY". 
Step 2: Applying other conditions
Once the match is identified, next step is to identify other conditions from user question and apply it to the identified matches.
Refe

In [39]:
%%time
question = "show me sales which are greater than 100"
output_measure=gereneate_output_openchat(compose_prompt_open_chat(question,MEASURE_PROMPT,context=MEASURE_LIST))
print("*******************MEASURE PROMPT output******************")
output_ = json.loads(output_measure.split("[SOL]")[-1].split("[/SOL]")[0])
print(output_)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


*******************MEASURE PROMPT output******************
[{'ENTITY': 'Sales', 'COMPARISON OPERATOR': '>', 'COMPARISON VALUE': '100'}]
CPU times: user 9.34 s, sys: 1.78 s, total: 11.1 s
Wall time: 11.1 s


In [40]:
%%time
question = "show me purchase value lower than 10k and higher than 5k"
output_measure=gereneate_output_openchat(compose_prompt_open_chat(question,MEASURE_PROMPT,context=MEASURE_LIST))
print("*******************MEASURE PROMPT output******************")
output_ = json.loads(output_measure.split("[SOL]")[-1].split("[/SOL]")[0])
print(output_)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


*******************MEASURE PROMPT output******************
[{'ENTITY': 'purchase value', 'COMPARISON OPERATOR': '<', 'COMPARISON VALUE': '10K'}, {'ENTITY': 'purchase value', 'COMPARISON OPERATOR': '>', 'COMPARISON VALUE': '5K'}]
CPU times: user 13.1 s, sys: 1.82 s, total: 14.9 s
Wall time: 14.9 s


In [41]:
%%time
question = "show me cars which are greater than 100"
output_measure=gereneate_output_openchat(compose_prompt_open_chat(question,MEASURE_PROMPT,context=MEASURE_LIST))
print("*******************MEASURE PROMPT output******************")
output_ = json.loads(output_measure.split("[SOL]")[-1].split("[/SOL]")[0])
print(output_)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


*******************MEASURE PROMPT output******************
[{'ENTITY': 'Quantity', 'COMPARISON OPERATOR': '>', 'COMPARISON VALUE': '100'}]
CPU times: user 9.33 s, sys: 1.86 s, total: 11.2 s
Wall time: 11.2 s


In [19]:
%%time
question = "discount rate of phone and binders"
output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,MEASURE_PROMPT,context=MEASURE_LIST))
print(output_measure.split("[_separator_]")[-2])

NameError: name 'tokenizer_mistral' is not defined

In [26]:
%%time
question = "show me sales greater than 100" 
output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
print(output_measure)
# output_measure_json = json.loads(output_measure.split("[_separator_]")[1])
# print(f"MEASURE_PROMPT output is : {output_measure_json}")
# print("MEASURE_PROMPT Done")
# context_data = find_best_match_from_context(output_measure_json['ENTITY'],context_df)
# Mql_measure= get_Mql(prompt_type="MEASURE",model_output=output_measure_json,context_data=context_data)
# print("--------MQL after post processing-----------")
# print(json.dumps(Mql_measure,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]<<SYS>>
            
Task: Extract Adjectives, Determine Intent (TONE), Return RANK ADJECTIVE, and Scalar (RANK)

Instructions:

Consider these list of dimension entities ['Sub-Category', 'sub-category', 'sub category', 'categories', 'section', 'Segment', 'segment', 'segments', 'units', 'divisions', 'Parts', 'parts', 'part', 'section', 'divisions', 'Country', 'country', 'countries'] as DIMENSION_ENTITIES_LIST
For each dimension entity detected in the give question, proceed to determine if there is any Adjective intent present within the question, Adjectives can include terms like "least" "highest", etc
Determine the intent (TONE) of the adjective. The intent can be only be either positive or negative.
If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
If the adjective intent is positive, always set RANK ADJECTIVE to "TOP" and TONE to "positive."
If the adjective implies a quantity (e.g., "how many," "top 3"), extract that numerical 

In [27]:
%%time
question = "show me sales greater than 100" 
output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
print(output_measure)
# output_measure_json = json.loads(output_measure.split("[_separator_]")[1])
# print(f"MEASURE_PROMPT output is : {output_measure_json}")
# print("MEASURE_PROMPT Done")
# context_data = find_best_match_from_context(output_measure_json['ENTITY'],context_df)
# Mql_measure= get_Mql(prompt_type="MEASURE",model_output=output_measure_json,context_data=context_data)
# print("--------MQL after post processing-----------")
# print(json.dumps(Mql_measure,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]<<SYS>>
            
Task: Extract Filter Entities
Analyze the provided question.
Consider only filter entity detected in the question, determine if it's intended to be included or excluded.
Set "EXCLUDE" to "True" if the detected filter entity is intended to be excluded else Set "EXCLUDE" to "False"

Return the only the following attributes for each detected filter entity in a json list:

{
"EXCLUDE": ("True" or "False"),
"ENTITY": (The specific filter entity matched from the FILTER_ENTITIES_LIST.)
}


            <</SYS>>
            
 Now detect from this question:
show me sales greater than 100

            [/INST]
            [_separator_]
            {
                "EXCLUDE": "False",
                "ENTITY": "sales"
            }
            [/_separator_]
CPU times: user 3min 55s, sys: 580 ms, total: 3min 56s
Wall time: 32.7 s


In [17]:
%%time
question = "show me purchase value lower than 10k and higher than 5k" 
output_measure=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,MEASURE_CONSTRAINT))
print(output_measure)
#output_measure_json = json.loads(output_measure.split("[_separator_]")[1])
# print(f"MEASURE_PROMPT output is : {output_measure_json}")
# print("MEASURE_PROMPT Done")
# context_data = find_best_match_from_context(output_measure_json['ENTITY'],context_df)
# Mql_measure= get_Mql(prompt_type="MEASURE",model_output=output_measure_json,context_data=context_data)
# print("--------MQL after post processing-----------")
# print(json.dumps(Mql_measure,indent=3))

NameError: name 'MEASURE_CONSTRAINT' is not defined

In [89]:
question = "discount rate of phone and binders"
output_filter = gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
output_filer_json = json.loads("["+f"""{output_filter.split("[_separator_]")[-2]}"""+ "]")
print(f"""FILETER_PROMPT output is : {output_filer_json}""")
Mql_filter = get_mql_filter(output_filer_json)
print("--------MQL after post processing-----------")
print(json.dumps(Mql_filter,indent=3))

FILETER_PROMPT output is : [{'EXCLUDE': 'False', 'ENTITY': 'discount rate'}, {'EXCLUDE': 'False', 'ENTITY': 'phone'}, {'EXCLUDE': 'False', 'ENTITY': 'binders'}]
--------MQL after post processing-----------
{
   "FILTER": {
      "discount rate": [
         {
            "ENTITY": "Discount",
            "EXCLUDE": "False",
            "PARENT": "nan"
         }
      ],
      "phone": [
         {
            "ENTITY": "Phone",
            "EXCLUDE": "False",
            "PARENT": "Sub-Category"
         }
      ],
      "binders": [
         {
            "ENTITY": "Binder",
            "EXCLUDE": "False",
            "PARENT": "Sub-Category"
         }
      ]
   }
}


In [95]:
%%time
question = "discount rate of phone and binders"
output_filter = gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
output_filer_json = json.loads("["+f"""{output_filter.split("[_separator_]")[-2]}"""+ "]")
print(f"""FILETER_PROMPT output is : {output_filer_json}""")
Mql_filter = get_mql_filter(output_filer_json)
print("--------MQL after post processing-----------")
print(json.dumps(Mql_filter,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


FILETER_PROMPT output is : [{'EXCLUDE': 'False', 'ENTITY': 'discount rate'}, {'EXCLUDE': 'False', 'ENTITY': 'phone'}, {'EXCLUDE': 'False', 'ENTITY': 'binders'}]
--------MQL after post processing-----------
{
   "FILTER": {
      "discount rate": [
         {
            "ENTITY": "Discount",
            "EXCLUDE": "False",
            "PARENT": "nan"
         }
      ],
      "phone": [
         {
            "ENTITY": "Phone",
            "EXCLUDE": "False",
            "PARENT": "Sub-Category"
         }
      ],
      "binders": [
         {
            "ENTITY": "Binder",
            "EXCLUDE": "False",
            "PARENT": "Sub-Category"
         }
      ]
   }
}
CPU times: user 5min 6s, sys: 293 ms, total: 5min 7s
Wall time: 42.3 s


In [96]:
%%time
question = "quantity across segments except consumer" 
output_filter = gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT_Context))
#output_filer_json = json.loads("["+f"""{output_filter.split("[_separator_]")[-2]}"""+ "]")
print(f"""FILETER_PROMPT output is : {output_filter}""")
# print("FILETER_PROMPT Done")
# mql_filter = get_mql_filter(filter_model_output)
# print("--------MQL after post processing-----------")
# print(json.dumps(mql_filter,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


FILETER_PROMPT output is : [INST]<<SYS>>
            
Task: Extract Filter Entities
Consider these list of filter entities ['Consumer', 'consumers', 'consumer', 'Phone', 'phone', 'phones', 'mobile phones', 'Binder', 'binders', 'binder', 'Corporate', 'corporates', 'corporate', 'India', 'india', 'Dubai', 'dubai'] as FILTER_ENTITIES_LIST.
Analyze the provided question.
Consider only filter entity detected in the question from FILTER_ENTITIES_LIST, determine if it's intended to be included or excluded.
Set "EXCLUDE" to "True" if the detected filter entity is intended to be excluded else Set "EXCLUDE" to "False"

Return the only the following attributes for each detected filter entity in a json list:

{
"EXCLUDE": ("True" or "False"),
"ENTITY": (The specific filter entity matched from the FILTER_ENTITIES_LIST.)
}


            <</SYS>>
            
 Now detect from this question:
quantity across segments except consumer

            [/INST]
            [_separator_]

Task: Extract Filter En

In [98]:
%%time
question = "discount rate of phone and binders"
output_filter = gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT_Context))
#output_filer_json = json.loads("["+f"""{output_filter.split("[_separator_]")[-2]}"""+ "]")
print(f"""FILETER_PROMPT output is : {output_filter} """)
# print("FILETER_PROMPT Done")
# mql_filter = get_mql_filter(filter_model_output)
# print("--------MQL after post processing-----------")
# print(json.dumps(mql_filter,indent=3))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


FILETER_PROMPT output is : [INST]<<SYS>>
            
Task: Extract Filter Entities
Consider these list of filter entities ['Consumer', 'consumers', 'consumer', 'Phone', 'phone', 'phones', 'mobile phones', 'Binder', 'binders', 'binder', 'Corporate', 'corporates', 'corporate', 'India', 'india', 'Dubai', 'dubai'] as FILTER_ENTITIES_LIST.
Analyze the provided question.
Consider only filter entity detected in the question from FILTER_ENTITIES_LIST, determine if it's intended to be included or excluded.
Set "EXCLUDE" to "True" if the detected filter entity is intended to be excluded else Set "EXCLUDE" to "False"

Return the only the following attributes for each detected filter entity in a json list:

{
"EXCLUDE": ("True" or "False"),
"ENTITY": (The specific filter entity matched from the FILTER_ENTITIES_LIST.)
}


            <</SYS>>
            
 Now detect from this question:
discount rate of phone and binders

            [/INST]
            [_separator_]

Task: Extract Filter Entities

In [115]:
output_dimension_json

{'ADJECTIVE': 'top',
 'TONE': 'positive',
 'ENTITY': 'segments',
 'RANK ADJECTIVE': 'TOP',
 'RANK': None}

In [114]:
question = "show me the list of top performing segments"
#output_dimension=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))

output_dimension_json = json.loads(output_dimension.split("[_separator_]")[1])
print(f"MEASURE_PROMPT output is : {output_dimsion_json}")
print("MEASURE_PROMPT Done")
context_data = find_best_match_from_context(output_dimension_json['ENTITY'],context_df)
Mql_dimension= get_Mql(prompt_type="DIMENSION",model_output=output_dimension_json, context_data=context_data)
print("--------MQL after post processing-----------")
print(json.dumps(Mql_dimension,indent=3))

MEASURE_PROMPT output is : {'ADJECTIVE': 'top', 'TONE': 'positive', 'ENTITY': 'segments', 'RANK ADJECTIVE': 'TOP', 'RANK': None}
MEASURE_PROMPT Done
--------MQL after post processing-----------
{
   "DIMENSION": {
      "segments": [
         {
            "ENTITY": "Consumer",
            "ADJECTIVE": " top ",
            "TONE": "positive",
            "RANK": [
               {
                  "RANK ADJECTIVE": "TOP",
                  "RANK VALUE": "None"
               }
            ]
         }
      ]
   }
}


In [118]:
question = "show me the highest top 10 sales in India"
output_dimension=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
output_dimension_json = json.loads(output_dimension.split("[_separator_]")[1])
print(f"MEASURE_PROMPT output is : {output_dimension}")
print("MEASURE_PROMPT Done")
# context_data = find_best_match_from_context(output_dimension_json['ENTITY'],context_df)
# Mql_dimension= get_Mql(prompt_type="DIMENSION",model_output=output_dimension_json, context_data=context_data)
# print("--------MQL after post processing-----------")
# print(json.dumps(Mql_dimension,indent=3))

MEASURE_PROMPT output is : [INST]<<SYS>>
            
Task: Extract Adjectives, Determine Intent (TONE), Return RANK ADJECTIVE, and Scalar (RANK)

Instructions:

Consider these list of dimension entities ['Sub-Category', 'sub-category', 'sub category', 'categories', 'section', 'Segment', 'segment', 'segments', 'units', 'divisions', 'Parts', 'parts', 'part', 'section', 'divisions', 'Country', 'country', 'countries'] as DIMENSION_ENTITIES_LIST
For each dimension entity detected in the give question, proceed to determine if there is any Adjective intent present within the question, Adjectives can include terms like "least" "highest", etc
Determine the intent (TONE) of the adjective. The intent can be only be either positive or negative.
If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
If the adjective intent is positive, always set RANK ADJECTIVE to "TOP" and TONE to "positive."
If the adjective implies a quantity (e.g., "how many," "top 3

In [ ]:
%%time
question = "show me the 2 top segments basis sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
%%time
question = "sales of Phone in Dubai"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




            **Task: Extract Filter Information**

Identify the filter entities in the user's question and set their inclusion or exclusion . For each filter entity mentioned in the question, check if it's intended to be included or excluded.

Return the following attributes for each detected filter entity:

EXCLUDE: Set to TRUE if the entity is intended to be excluded, FALSE otherwise.
ENTITY: The specific filter entity identified from the question.

If the question specifies exclusion for an entity, set EXCLUDE to TRUE; otherwise, set it to FALSE.

**Return:**
- EXCLUDE: TRUE or FALSE
- ENTITY: The filter entity
            [/ _separator_]

            **Filter Entities:**

            - ENTITY: Phone
            - ENTITY: Dubai

            **Filter Inclusion:**

            - EXCLUDE: FALSE
            - ENTITY: Phone
            - EXCLUDE: FALSE
            - ENTITY: Dubai
CPU times: user 8min 40s, sys: 808 ms, total: 8min 41s
Wall time: 1min 8s


In [17]:
question = "sales of segements except in US"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




            **Task: Extract Filter Information**

Identify the filter entities in the user's question and set their inclusion or exclusion . For each filter entity mentioned in the question, check if it's intended to be included or excluded.

Return the following attributes for each detected filter entity:

EXCLUDE: Set to TRUE if the entity is intended to be excluded, FALSE otherwise.
ENTITY: The specific filter entity identified from the question.

If the question specifies exclusion for an entity, set EXCLUDE to TRUE; otherwise, set it to FALSE.

**Return:**
- EXCLUDE: TRUE or FALSE
- ENTITY: The filter entity
            [/separator_]

            **Filter Entities:**

            - sales
            - segments
            - US

            **Filter Information:**

            - sales: EXCLUDE: FALSE, ENTITY: sales
            - segments: EXCLUDE: FALSE, ENTITY: segments
            - US: EXCLUDE: TRUE, ENTITY: US


In [49]:
%%time
question = "show me quantities > 100K"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATOR" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "show me quantities > 100K"

**Output**:
- COMPARISON OPERATOR: greater than
- COMPAR

In [50]:
%%time
question = "When is first time discount of phone is lesser than 0"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATORS" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "When is first time discount of phone is lesser than 0"

**Output**:
- COMPARISON OP

In [8]:
%%time
question = "top 2 segments and bottom 3 sub-category basis quantity"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)




**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "top," "bottom," "best," "worst," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "top 2 segments and bottom 3 sub-category basis quantity"

**Output**:
- ADJECTIVE: top, bottom
- TONE: mixed (positive and negative)
- ENTITY: Segment, sub-category
- RANK ADJECTIVE: TOP, BOTTOM
- TONE

In [12]:
%%time
question = "show me the 2 top segments basis sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)




**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "least," "highest performing," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "show me the 2 top segments basis sales"

**Output**:
- ADJECTIVE: top
- TONE: positive
- ENTITY: Segment
- RANK ADJECTIVE: TOP
- TONE: positive
- RANK: 2
CPU times: user 18.8 s, sys: 602 ms, total: 19.4 s

In [14]:
ADJECTIVE_and_TONE_output = post_process_model_output(output)
ADJECTIVE_and_TONE_output

{'ADJECTIVE': 'top',
 'TONE': 'positive',
 'ENTITY': 'Segment',
 'RANK ADJECTIVE': 'TOP',
 'RANK': '2'}

In [12]:
{
    "DIMENSION": {
        "segments": [
            {
                "ENTITY": "Segment",
                "RANK": [
                    {
                        "RANK ADJECTIVE": "top",
                        "RANK VALUE": "2"
                    }
                ]
            }
        ]
    },
    "MEASURE": {
        "sales": [
            {
                "ENTITY": "Sales"
            }
        ]
    }
}


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'}]}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [16]:
#!pip install fuzzywuzzy

In [11]:
%%time
question = "top 2 and bottom 3 segments by sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.60 GiB total capacity; 14.56 GiB already allocated; 14.94 MiB free; 14.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF